# Dataset: BinaryClassification
# Algorith: Logistic Regression

1. Importing files and modules:

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
!pip install gdown
import gdown
#this way, anyone can access the files without mounting google driving. Accessing the csv files:
csvFileID = '1GZEf5NV8rX1xbqfaoOp0qrnhzyBQLZ0I'
url = f'https://drive.google.com/uc?id={csvFileID}'
csvOutputFile = 'binary_classification_train.csv'
gdown.download(url, csvOutputFile, quiet=False)

#accessing the module files:
moduleFileID = '18FkmyTFD_ooChnvtCrLSToRAd09Nkwr2'
url = f'https://drive.google.com/uc?id={moduleFileID}'
moduleOutputFile = 'LogisticRegression.py'
gdown.download(url, moduleOutputFile, quiet=False)
from LogisticRegression import LogisticRegression
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Downloading...
From: https://drive.google.com/uc?id=1GZEf5NV8rX1xbqfaoOp0qrnhzyBQLZ0I
To: /content/binary_classification_train.csv
100%|██████████| 18.6M/18.6M [00:00<00:00, 19.8MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=18FkmyTFD_ooChnvtCrLSToRAd09Nkwr2
From (redirected): https://drive.google.com/uc?id=18FkmyTFD_ooChnvtCrLSToRAd09Nkwr2&confirm=t&uuid=8096bbd7-044c-4827-bdc3-47d0314834f0
To: /content/LogisticRegression.py
100%|██████████| 10.1k/10.1k [00:00<00:00, 18.6MB/s]


2. Using pandas to read the CSV file:

In [ ]:
data = pd.read_csv(csvOutputFile)

In [ ]:
data.head()

,ID,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,...,Feature_12,Feature_13,Feature_14,Feature_15,Feature_16,Feature_17,Feature_18,Feature_19,Feature_20,Class
0,1,-36.963099,-15.014703,87.100435,101.523360,82.106571,-10.956180,0.592326,-51.919028,-90.650434,...,-81.183744,13.101921,18.051450,-40.606288,-39.697053,-13.870802,173.722987,-17.349169,-82.550844,0
1,2,-43.715674,18.847116,89.543406,-71.319314,35.597052,126.358570,-29.837495,-40.473764,-94.079238,...,-37.848260,7.457352,-77.420742,53.773718,-100.124294,9.870650,11.592519,-106.123605,-92.796421,1
2,3,-30.737550,-63.729643,106.081332,81.773948,112.769976,-12.425351,-29.913286,-41.771200,16.424511,...,-106.610289,5.930143,-30.177083,-138.969234,-56.054914,-12.790661,164.832498,-37.412902,-85.441150,0
3,4,-27.674757,-118.869495,135.605213,99.130189,50.947548,-63.704785,-7.353057,-58.140229,-80.209027,...,-149.056417,3.893419,-74.100869,-47.659832,-48.209817,-36.264323,59.001922,-59.064134,-78.538639,0
4,5,-28.654141,-77.746597,85.215365,50.374774,79.763207,-32.703048,-28.152031,-63.994794,-153.566789,...,-102.731465,11.160205,-13.395073,9.416237,2.649524,-43.578704,52.261888,-66.081738,-80.325110,0


3. Extracting X and Y from the Pandas DataFrame:

In [ ]:
feature_col = [f"Feature_{i}" for i in range(1,21)]
X = data[feature_col].to_numpy()
Y = data['Class'].to_numpy()
IDs = data['ID'].to_numpy()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

class LogisticRegression:
    def __init__(self,inp_features, alpha=1e-3, reg_param=None, batch_size=32,epochs=100):
        self.epochs = epochs
        self.inp_features = inp_features
        self.alpha = alpha
        self.reg_param = reg_param
        self.W = None
        self.B = None
        self._init_params()
        self.batch_size = batch_size
        self.thresh = 0.5
    def _init_params(self):
        self.W = np.random.randn(1,self.inp_features)
        self.B = 0.0
    def _compute_cost(self, X,Y):
        m = Y.size
        cost = -(1/m)*np.sum(Y*np.log(self.predict_fwb(X)))
        return cost
    def _add_reg_term(self,m):
        reg_cost = np.sum(self.W**2)
        reg_cost = (reg_cost*self.reg_param)/(2*m)
        return reg_cost
    def _get_grads(self,X,Y):
        predictions = self.predict_fwb(X)
        err = predictions - Y
        dj_dw = np.mean(np.multiply(err, X), axis=0)
        dj_db = np.mean(err)
        return dj_dw, dj_db
    def train(self, X, Y, details=False,plot_costs=True,get_report=True,print_con_mat=True):
        m = Y.size
        Y = Y.reshape((m,1))
        J_history_batches = []
        J_hist_ep = []
        for epoch in range(1, self.epochs+1):
            permutation = np.random.permutation(m)
            X_shuffled = X[permutation,:]
            Y_shuffled = Y[permutation,:]
            batches = m //self.batch_size
            for k in range(batches):
                mini_batch_X = X_shuffled[k*self.batch_size:(k+1)*self.batch_size,:]
                mini_batch_Y = Y_shuffled[k*self.batch_size: (k+1)*self.batch_size,:]
                dj_dw, dj_db = self._get_grads(mini_batch_X, mini_batch_Y)
                if self.reg_param is None:
                  self.W -= self.alpha*dj_dw
                  self.B -= self.alpha*dj_db
                else:
                  self.W = self.W*(1-self.alpha*self.reg_param/m)
                  self.W -= self.alpha*dj_dw
                  self.B -= self.alpha*dj_db
                cost = self._compute_cost(mini_batch_X, mini_batch_Y)
                J_history_batches.append(cost)
                cost += self._add_reg_term(mini_batch_Y.size) if self.reg_param is not None else 0
                if details:
                    print(f"Epoch: {epoch:03d}, Batch: {k+1}/{batches}, Cost: {cost:.6f}")

            if m%batches != 0:
                mini_batch_X = X_shuffled[batches*self.batch_size:,:]
                mini_batch_Y = Y_shuffled[batches*self.batch_size]
                dj_dw, dj_db = self._get_grads(mini_batch_X, mini_batch_Y)
                if self.reg_param is None:
                  self.W -= self.alpha*dj_dw
                  self.B -= self.alpha*dj_db
                else:
                  self.W = self.W*(1-self.alpha*self.reg_param/m)
                  self.W -= self.alpha*dj_dw
                  self.B -= self.alpha*dj_db

                cost = self._compute_cost(mini_batch_X, mini_batch_Y)
                J_history_batches.append(cost)
                cost += self._add_reg_term(mini_batch_Y.size) if self.reg_param is not None else 0
                if details:
                    print(f"Epoch: {epoch:03d}, Batch: last, Cost: {cost:.6f}")

            cost = self._compute_cost(X_shuffled, Y_shuffled)
            J_hist_ep.append(cost)
            cost += self._add_reg_term(m) if self.reg_param is not None else 0
            print(f"Epoch: {epoch:03d}, Cost: {cost:.6f}, Accuracy: {self.get_accuracy(Y_shuffled, self.predict(X_shuffled)):.4f}")

        if print_con_mat:
            predictions = self.predict(X)
            con_mat = self.get_conf_mat(Y, predictions)
            self.print_con_mat(con_mat)
        if get_report:
            predictions = self.predict(X)
            self.classification_report(Y, predictions)
        self._plotter(J_hist_ep) if plot_costs else None
        return J_hist_ep
    def k_fold_cv(self, X, Y, k=5, details=False,plot_cost_vs_epoch=True, plot_cost_vs_acc=True,retrain=False):
        m = X.shape[0]
        indxs = np.random.permutation(m)
        X_shuffled = X[indxs,:]
        Y_shuffled = Y[indxs]
        testing_accs = []
        training_accs = []
        if plot_cost_vs_epoch:
            J_hist_list = []
        fold_size = m // k
        for fold in range(1,k+1):
            self._init_params()
            print(f"Working on fold {fold}....")
            test_start = (fold-1)*fold_size
            test_end = fold*fold_size if fold < k-1 else m
            X_train = np.concatenate((X_shuffled[:test_start,:], X_shuffled[test_end:,:]), axis=0)
            Y_train = np.concatenate((Y_shuffled[:test_start], Y_shuffled[test_end:]))
            X_test = X_shuffled[test_start:test_end,:]
            Y_test = Y_shuffled[test_start:test_end]
            X_train_mean = np.mean(X_train, axis=0)
            X_train_std = np.std(X_train, axis=0)
            X_train = (X_train - X_train_mean)/X_train_std
            X_test = (X_test - X_train_mean)/X_train_std
            J_hist_ep = self.train(X_train, Y_train,details=details,plot_costs=False,get_report=False,print_con_mat=False)
            print(f"Training Report for fold {fold}:")
            self.classification_report(Y_train, self.predict(X_train))
            J_hist_list.append(J_hist_ep)
            print(f"Testing report for fold {fold}: ")
            self.classification_report(Y_test, self.predict(X_test))
            test_acc = self.get_accuracy(Y_test, self.predict(X_test))
            train_acc = self.get_accuracy(Y_train, self.predict(X_train))
            testing_accs.append(test_acc)
            training_accs.append(train_acc)
            print(f"Fold {fold} completed!")
            print(f"Starting fold {fold +1}") if fold<k else None
        mean_test_acc = np.mean(testing_accs)
        mean_train_acc = np.mean(training_accs)
        print(f"Mean of training Accuracies is {mean_train_acc}")
        print(f"Mean of testing accuracies: {mean_test_acc}")
        if plot_cost_vs_epoch:
            for fold in range(1,k+1):
                plt.plot(np.arange(1,1+len(J_hist_list[fold-1])), J_hist_list[fold-1], c='r',label='Cost')
                plt.xlabel("Epochs")
                plt.ylabel("Costs")
                plt.title(f"Costs vs Epochs for fold {fold}")
                plt.legend()
                plt.show()

        if plot_cost_vs_acc:
            plt.plot(np.arange(1,k+1), training_accs, c='blue', label='Training Accuracies', linestyle='--', marker='o')
            plt.plot(np.arange(1,k+1), testing_accs, c='green', label='Testing Accuracies', linestyle='--', marker='o')
            plt.xlabel("Folds")
            plt.ylabel("Accuracies")
            plt.title("Folds vs Accuracies")
            plt.legend()
            plt.show()

        return mean_train_acc, mean_test_acc, training_accs, testing_accs
    def _sigmoid(self, z):
        return 1/(1+np.exp(-z))
    def predict_fwb(self, X):
        z =  np.dot(X,self.W.T) + self.B
        return self._sigmoid(z)
    def predict(self, X):
        predictions = self.predict_fwb(X)
        pred_final = (predictions >= self.thresh).astype(int)
        return pred_final
    def _plotter(self, J_hist):
        plt.plot(np.arange(1,len(J_hist)+1), J_hist, c='r')
        plt.xlabel('Epochs')
        plt.ylabel('Cost')
        plt.title("Cost vs Epochs")
        plt.show()

    def get_conf_mat(self, Y_act, Y_pred):
        n_cls = len(np.unique(Y_act))
        con_mat = np.zeros((n_cls, n_cls))
        for indx, label in enumerate(Y_act):
            con_mat[label, Y_pred[indx]] += 1

        return con_mat

    def print_con_mat(self, con_mat):
        print("Confusion matrix with predictions on X axis and actual values on Y")
        classes= np.arange(con_mat.shape[0])
        table = pd.DataFrame(con_mat, index=classes, columns=classes)
        print(table)

    def get_precision(self,con_mat):
        precs = np.zeros((con_mat.shape[0],))
        for i in range(con_mat.shape[0]):
            precs[i] = con_mat[i,i]/np.sum(con_mat[:,i])

        return precs

    def get_recall(self, con_mat):
        recs = np.zeros((con_mat.shape[0],))
        for i in range(con_mat.shape[0]):
            recs[i] = con_mat[i,i]/np.sum(con_mat[i,:])

        return recs

    def get_f1s(self, con_mat):
        precs = self.get_precision(con_mat)
        recs = self.get_recall(con_mat)
        f1s = 2*recs*precs/(recs + precs)
        return f1s
    def get_accuracy(self, Y_act, Y_pred):
        return 100*np.mean(Y_act == Y_pred)
    def classification_report(self, Y_act, Y_pred):
        con_mat = self.get_conf_mat(Y_act, Y_pred)
        precs = self.get_precision(con_mat)
        recs = self.get_recall(con_mat)
        f1s = self.get_f1s(con_mat)
        supports = con_mat.sum(axis=1)
        classes= np.arange(con_mat.shape[0])
        reports = []
        for i in range(con_mat.shape[0]):
            reports.append([precs[i], recs[i], f1s[i], supports[i]])
        table1 = pd.DataFrame(reports,index=classes, columns=['precision','recall','f1-score','support'])
        # print(table1)
        wted_prec = np.average(precs, weights=supports)
        wted_rec = np.average(recs, weights=supports)
        wted_f1 = np.average(f1s, weights=supports)
        tot_support = np.sum(supports)
        wted_data = [[wted_prec,wted_rec,wted_f1,tot_support]]
        table2 = pd.DataFrame(wted_data,index=['Average (weighted)'],columns=['precision', 'recall', 'f1-score', 'support'])
        # print('\n\n', table2)
        table = pd.concat([table1, table2])
        print(table)
        print(f"Accuracy: {self.get_accuracy(Y_pred, Y_act)}%")

4. Standardizing X. Then training model with alpha = 1e-3, L2 regularization parameter (lamba) = 1e-2 and epochs=500

In [ ]:
X_mean = np.mean(X, axis=0)
X_std = np.mean(X, axis=0)
X = (X - X_mean)/X_std
model = LogisticRegression(20, alpha=1e-3,reg_param=1e-2, epochs=500)
model.train(X, Y, details=False, plot_costs=True, get_report=True,print_con_mat=True)

5. Importing the test data:

In [ ]:
YFileID = '19Wwiu1CMGuF4p26aDtlVQjb-8b0x3Lpa'
url = f'https://drive.google.com/uc?id={YFileID}'
HiddenData = 'binary_classification_test.csv'
gdown.download(url, HiddenData, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=19Wwiu1CMGuF4p26aDtlVQjb-8b0x3Lpa
To: /content/binary_classification_test.csv
100%|██████████| 4.61M/4.61M [00:00<00:00, 123MB/s]


'binary_classification_test.csv'

In [ ]:
df = pd.read_csv(HiddenData)

6. Standardizing the X_data (z score)

In [ ]:
X_data = df[feature_col].to_numpy()
X_data = (X_data - X_mean)/X_std

7. Making predictions and then exporting as CSV file with ID vs Targets

In [ ]:
mapping = []
predictions = model.predict(X_data)
for i in range(X_data.shape[0]):
  mapping.append([i+1,predictions[i]])

new_df = pd.DataFrame(mapping, columns=['ID', 'Class'])
new_df.sort_values(by='ID', inplace=True)
print(new_df.head())
new_df.to_csv('binaryMapping_LR.csv', index=False)

   ID Class
0   1   [0]
1   2   [0]
2   3   [0]
3   4   [0]
4   5   [0]
